In [0]:
# import relevant libraries
%matplotlib inline
import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# confirm TensorFlow can see the GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# load Fashion dataset

from google.colab import files

uploaded = files.upload()

'''
files.upload() returns a dictionary of the files which were uploaded. The 
dictionary is keyed by the file name, the value is the data which was 
uploaded

'''

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn,length=len(uploaded[fn])))

In [0]:
# load MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

In [0]:
'''
Create inputs for our graph. We'll have two inputs - one for the 
generator (input_z) and one for the discriminator(inputs_real).

'''
def model_inputs(real_dim, z_dim):
  inputs_real = tf.placeholder(tf.float32, (None, real_dim), name='input_real')
  inputs_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
  
  return inputs_real, inputs_z

In [0]:
def generator(z,out_dim,n_units=128,reuse=False,alpha=0.01):
  with tf.variable_scope('generator', reuse=reuse):
    # Hidden layer
    h1 = tf.layers.dense(z,n_units,activation=None)
    # Leaky ReLU
    h1 = tf.maximum(alpha*h1,h1)
    
    # Logits and tanh output
    logits = tf.layers.dense(h1,out_dim,activation=None)
    out = tf.tanh(logits)
    
    return out

In [0]:
def discriminator(x,n_units=128,reuse=False,alpha=0.01):
  with tf.variable_scope('discriminator',reuse=reuse):
    # Hidden layer
    h1 = tf.layers.dense(x,n_units,activation=None)
    # Leaky ReLU
    h1 = tf.maximum(alpha*h1,h1)
    
    # Logits and sigmoid output
    logits = tf.layers.dense(h1,1,activation=None)
    out = tf.sigmoid(logits)
    
    return out,logits

In [0]:
''' HYPERPARAMETERS'''

# Size of input image to discriminator
input_size = 784
# Size of latent vector to generator
z_size = 100
# Sizes of hidden layers in generator and discriminator
g_hidden_size = 128
d_hidden_size = 128
# Leak factor for leaky ReLU
alpha = 0.01
# Smoothing
smooth = 0.1

In [0]:
''' BUILDING NETWORK'''
tf.reset_default_graph()
# Create our input placeholders
input_real, input_z = model_inputs(input_size, z_size)

# Build model, g_model is generator output
g_model = generator(input_z, input_size, n_units=g_hidden_size, alpha=alpha)

d_model_real, d_logits_real = discriminator(input_real, n_units=d_hidden_size, alpha=alpha)
d_model_fake, d_logits_fake = discriminator(g_model, reuse=True, n_units=d_hidden_size,alpha=alpha)

In [18]:
''' CTrlv'''

# Calculate losses
d_loss_real = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, 
                                                          labels=tf.ones_like(d_logits_real) * (1 - smooth)))
d_loss_fake = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, 
                                                          labels=tf.zeros_like(d_logits_real)))
d_loss = d_loss_real + d_loss_fake

g_loss = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                     labels=tf.ones_like(d_logits_fake)))
# Optimizers
learning_rate = 0.002

# Get the trainable_variables, split into G and D parts
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)


batch_size = 100
epochs = 100
samples = []
losses = []
# Only save generator variables
saver = tf.train.Saver(var_list=g_vars)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            
            # Get images, reshape and rescale to pass to D
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2 - 1
            
            # Sample random noise for G
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            
            # Run optimizers
            _ = sess.run(d_train_opt, feed_dict={input_real: batch_images, input_z: batch_z})
            _ = sess.run(g_train_opt, feed_dict={input_z: batch_z})
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_real: batch_images})
        train_loss_g = g_loss.eval({input_z: batch_z})
            
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g))    
        # Save losses to view after training
        losses.append((train_loss_d, train_loss_g))
        
        # Sample from generator as we're training for viewing afterwards
        sample_z = np.random.uniform(-1, 1, size=(16, z_size))
        gen_samples = sess.run(
                       generator(input_z, input_size, n_units=g_hidden_size, reuse=True, alpha=alpha),
                       feed_dict={input_z: sample_z})
        samples.append(gen_samples)
        saver.save(sess, './checkpoints/generator.ckpt')

# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)


Epoch 1/100... Discriminator Loss: 0.3653... Generator Loss: 3.9091
Epoch 2/100... Discriminator Loss: 0.3904... Generator Loss: 3.7598
Epoch 3/100... Discriminator Loss: 0.4889... Generator Loss: 3.2077
Epoch 4/100... Discriminator Loss: 0.7346... Generator Loss: 3.1952
Epoch 5/100... Discriminator Loss: 3.2319... Generator Loss: 2.5451
Epoch 6/100... Discriminator Loss: 1.6714... Generator Loss: 1.6889
Epoch 7/100... Discriminator Loss: 1.8072... Generator Loss: 1.4677
Epoch 8/100... Discriminator Loss: 0.9690... Generator Loss: 1.5453
Epoch 9/100... Discriminator Loss: 0.7293... Generator Loss: 3.1458
Epoch 10/100... Discriminator Loss: 1.8963... Generator Loss: 1.8863
Epoch 11/100... Discriminator Loss: 0.7792... Generator Loss: 2.0302
Epoch 12/100... Discriminator Loss: 0.9301... Generator Loss: 2.3031
Epoch 13/100... Discriminator Loss: 1.4263... Generator Loss: 1.9884
Epoch 14/100... Discriminator Loss: 1.5954... Generator Loss: 1.5664
Epoch 15/100... Discriminator Loss: 1.2234.